# Library & Setup

###### Package installation

In [5]:
pip install snowflake-connector-python[pandas]

  Created wheel for ijson: filename=ijson-2.5.1-cp37-none-any.whl size=16986 sha256=52819df12aeb9b96c0e4cce65a27e4d08369d704327f0e917e3745da0a04b5f0
  Stored in directory: C:\Users\hn42\AppData\Local\pip\Cache\wheels\4d\d1\00\97c35b66b6b59081fe7f90f4a4dc30673ebd8af70382238b95
Successfully built ijson
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Created wheel for pandas-profiling: filename=pandas_profiling-2.4.1-py2.py3-none-any.whl size=209427 sha256=4defc74418b93c6fcf9b64dc66d0aad1729b48ddc6bcb9058a6154afdcfcb106
  Stored in directory: C:\Users\hn42\AppData\Local\Temp\pip-ephem-wheel-cache-_13lb98k\wheels\56\c2\dd\8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
  Created wheel for confuse: filename=confuse-1.0.0-cp37-none-any.whl size=17494 sha256=34229ba94b0e3d3a8d2f3b9a60afdb16bf85013e078b8de5126f02b9672b9def
  Stored in directory: C:\Users\hn42\AppData\Local\pip\Cache\wheels\b0\b2\96\2074eee7dbf7b7df69d004c9b6ac4e32dad04fb7666cf943bd
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27090 sha256=cc2ae2ed40d37f1c6102da1854a535a8da26300bfc739e4044092d5e9216fabe
  Stored in directory: C:\Users\hn42\AppData\Local\pip\Cache\wheels\43\07\ac\7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built pandas-profiling confuse htmlmin
Note: you may need to restart the kernel to use updated 

In [115]:
pip install jinjasql

  Created wheel for jinjasql: filename=jinjasql-0.1.7-cp37-none-any.whl size=4758 sha256=4a30261c4f5e22bb6f492c90ac830c211aae263b2058013176289f1d2359121e
  Stored in directory: C:\Users\hn42\AppData\Local\pip\Cache\wheels\81\d6\f3\3aed06af06cfe7f53e72bc51c76e416097de87f32a361e34ab
Successfully built jinjasql
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install pandas-profiling

In [13]:
pip install git+https://github.com/sat28/githubcommit.git

  Cloning https://github.com/sat28/githubcommit.git to c:\users\harsha~1.nan\appdata\local\temp\pip-req-build-dnf9363l
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/sat28/githubcommit.git 'C:\Users\HARSHA~1.NAN\AppData\Local\Temp\pip-req-build-dnf9363l'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/sat28/githubcommit.git 'C:\Users\HARSHA~1.NAN\AppData\Local\Temp\pip-req-build-dnf9363l'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [ ]:
jupyter serverextension enable --py githubcommit
jupyter nbextension install --py githubcommit

###### Libraries & Pandas parameters

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

In [2]:
# set up display area to show dataframe in jupyter qtconsole
import pandas as pd
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 300)
#pd.set_option('display.height', 300)
#pd.display.float_format(.2)
#Display maximum column width
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

###### Snowflake Connector & Execute Functions

In [3]:
import snowflake.connector as sf
ctx=sf.connect(
    account="tailoredbrands.us-east-1",
    user="HARSHAVARDHAN.NANNUR@TAILOREDBRANDS.COM", #change user name 
    warehouse="ANALYTICS_MEDIUM", 
    database="GRP_LOOM",
    schema="BLT",
    authenticator='externalbrowser')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
def exec_sql(sql,ctx=ctx):
    df = pd.read_sql(sql, ctx)
    return df

# Running stats on SQL Columns

In [6]:
COLUMN_STATS_TEMPLATE = '''
select
    {{ column_name | sqlsafe }} as column_name
    , count(*) as num_rows
    , count(distinct {{ column_name | sqlsafe }}) as num_unique
    , sum(case when {{ column_name | sqlsafe }} is null then 1 else 0 end) as num_nulls
    {% if default_value %}
    , sum(case when {{ column_name | sqlsafe }} = {{ default_value }} then 1 else 0 end) as num_default
    {% else %}
    , 0 as num_default
    {% endif %}
    , min({{ column_name | sqlsafe }}) as min_value
    , max({{ column_name | sqlsafe }}) as max_value
from
    {{ table_name | sqlsafe }}
    group by column_name
'''


from copy import deepcopy
def get_sql_from_template(query, bind_params):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params


from jinjasql import JinjaSql
def apply_sql_template(template, parameters):
    '''
    Apply a JinjaSql template (string) substituting parameters (dict) and return
    the final SQL.
    '''
    j = JinjaSql(param_style='pyformat')
    query, bind_params = j.prepare_query(template, parameters)
    return get_sql_from_template(query, bind_params)

def get_column_stats_sql(table_name, column_name, default_value):
    '''
    Returns the SQL for computing column statistics.
    Passing None for the default_value results in zero output for the number
    of default values.
    '''
    params = {
        'table_name': table_name,
        'column_name': column_name,
        'default_value': default_value,
    }
    return apply_sql_template(COLUMN_STATS_TEMPLATE, params)

In [7]:
def col_stats(table_name,columns_list):
    sql_out=[]
    for i in columns_list:
        list_out=exec_sql(get_column_stats_sql(table_name,i,None))
        list_out['Col_Name']=i
        sql_out.append(list_out)
    df=pd.concat(sql_out)
    return df

In [208]:
Table_name='BLT.Order_detail_master'
columns=list(['Brand','ORDER_TYPE'])
df=col_stats(Table_name,columns)
df[df.Col_Name=='Brand']

,COLUMN_NAME,NUM_ROWS,NUM_UNIQUE,NUM_NULLS,NUM_DEFAULT,MIN_VALUE,MAX_VALUE,Col_Name
0,MSP,83841281,1,0,0,MSP,MSP,Brand
1,JAB,106765936,1,0,0,JAB,JAB,Brand
2,TMW,649401651,1,0,0,TMW,TMW,Brand
3,KNG,133418057,1,0,0,KNG,KNG,Brand


# Data Loading

In [61]:
sql="select * from ODS.EXPERIAN_CUSTOMER_SUMMARY sample row (10000 rows)" #getting a sample of 100,000 rows
ods_exp_sample_dt=exec_sql(sql)

In [86]:
sql="select * from BLT.Order_master sample row (100000 rows)" #getting a sample of 100,000 rows
blt_odr_mstr=exec_sql(sql)

In [10]:
sql="select * from spa.COMBINED_SURVEY_DATA"
survey_data=exec_sql(sql)

In [27]:
Quan=pd.read_csv('quan_NTF.csv')

# Profiling ( Options)

###### Option-1 Small data & Detailed statistics 

In [87]:
profile = pandas_profiling.ProfileReport(blt_odr_dtl_mstr)
profile.to_file(output_file="odr_dtl_mstr.html")

C:\Users\hn42\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:35: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr / min((kcorr - 1.0), (rcorr - 1.0)))
C:\Users\hn42\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:126: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 0).')
  correlation_name=correlation_name, error=error


###### Option-2 Large data and limited statistics

In [ ]:
#Version-1 ( Still takes lot of time to run)
profile = surve_data.profile_report(correlations={
    "pearson": False,
    "spearman": False,
    "kendall": False,
    "phi_k": False,
    "cramers": False,
    "recoded":False,}
)
profile.to_file(outputfile="output.html")

In [ ]:
#Version-2 ( Working and runs fast)
profile = pandas_profiling.ProfileReport(survey_data, minimal=True)
profile.to_file(output_file="output2.html")

## Analysis Starts Here

In [11]:
survey_data.columns

Index(['SURVEY', 'DATE', 'TIME', 'RESPONDENTID', 'SATISFACTION_OVERALL', 'RECOMMEND_COMPANY', 'BRAND', 'CUSTOM_ORDER_ID', 'TRANSACTION_ID', 'SURVEY1', 'CUSTOMER_NUM'], dtype='object')